# Tutorial - Model

We now illustrate the basic capabilities of the ``respy`` package in a simple tutorial.

## The model specification

In order to perform simulation and/or estimation using ``respy`` package a model specification is needed. It consists of two files: the parameter specification contains initial parameter values and the options specification includes important data set dimentions, arguments for the optimization algorythms, etc.. Details on the components of the model specification are presented in the section [Model specification](../software/model-specification.rst). Please note that the two specification files should be in your current working directory or another accessible directory when executing the commands and scripts discussed below.

## Example

Now we can explore the basic functionalities of the ``respy`` package based on a simple example.

In [1]:
import respy as rp

In [2]:
# Get an exemplary model specification.
params_spec, options_spec = rp.get_example_model("kw_data_one")

The options specification shows all variable arguments to the model which are not part of the optimization process like the number of periods in the model or the optimizer to fit the model to data.

In [3]:
options_spec

{'estimation': {'agents': 1000, 'draws': 200, 'seed': 500, 'tau': 500.0},
 'simulation': {'agents': 1000, 'seed': 132},
 'program': {'debug': False},
 'interpolation': {'flag': False, 'points': 200},
 'solution': {'seed': 456, 'draws': 500},
 'edu_spec': {'lagged': [1.0], 'start': [10], 'share': [1.0], 'max': 20},
 'num_periods': 40}

In [4]:
# We limit the model to five periods to make runtime shorter
# and to avoid memory errors on mybinder.org.
options_spec["num_periods"] = 5

The parameter specification includes all parameters of the model which are affected by the optimization routine.

In [5]:
params_spec.head(5)

,category,name,para,fixed,lower,upper,comment
0,delta,delta,0.950,False,0.7,1.0,discount factor
1,coeffs_common,return_hs_degree,0.000,False,NaN,NaN,return to high school degree (non pecuniary)
2,coeffs_common,return_col_degree,0.000,False,NaN,NaN,return to college degree (non pecuniary)
3,coeffs_a,skill_price,9.210,False,NaN,NaN,skill rental price if the base skill endowment...
4,coeffs_a,return_schooling,0.038,False,NaN,NaN,linear return to an additional year of schooli...


In [6]:
# Convert parameters and options to model attributes.
attr = rp.process_model_spec(params_spec, options_spec)

In [7]:
# Simulate a sample from the specified model.
state_space, df = rp.simulate(attr)

In [8]:
# Calculate the likelihood for the simulated sample.
crit_func = rp.get_criterion_function(attr, df)
x = rp.get_parameter_vector(attr)

crit_val = crit_func(x)
crit_val

2.0070985859621238

The simulation and estimation functionalities of the ``respy`` package can be used
separately. To perform a simulation only a model specification, as shown above, is
required. Here we are using the simulated data
for the estimation. However, you can of course also use other data sources.
For more information on the required structure of the dataset see
[Model specification](../software/model-specification.rst).